In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
# Starting dataset
df = pd.read_csv('customer_supermarket.csv', sep='\t', index_col=0)

In [3]:
# Handle NAN CustomerID rows
df.drop(df[~df['CustomerID'].notnull()].index, inplace=True)

In [4]:
# Set Sale to have the correct type
df['Sale'].replace(to_replace=r'(\d+),(\d*)', value=r'\1.\2', regex=True, inplace=True)
df['Sale'] = df['Sale'].astype(float)

In [5]:
# Clean CustomerID from trailing '.0'
df['CustomerID'] = df['CustomerID'].astype(str)
df['CustomerID'].replace(to_replace=r'(\d+)\.(?:\d*)?', value=r'\1', regex=True, inplace=True)

In [6]:
# Initialize variables to analyze correctness of negative Qta rows
pos_quant_tot = {}
neg_quant_tot = {}
for index, row in df.iterrows():
    if row['Qta'] > 0:
        if (row['CustomerID'], row['ProdID'], row['Sale']) not in pos_quant_tot.keys():
            pos_quant_tot[(row['CustomerID'], row['ProdID'], row['Sale'])] = row['Qta']
        else:
            pos_quant_tot[(row['CustomerID'], row['ProdID'], row['Sale'])] += row['Qta']
    else:
        if (row['CustomerID'], row['ProdID'], row['Sale']) not in neg_quant_tot.keys():
            l = [index]
            neg_quant_tot[(row['CustomerID'], row['ProdID'], row['Sale'])] = (-1*row['Qta'], l)
        else:
            prev = neg_quant_tot[(row['CustomerID'], row['ProdID'], row['Sale'])] 
            prev[1].append(index)
            neg_quant_tot[(row['CustomerID'], row['ProdID'], row['Sale'])] = (prev[0]-row['Qta'], prev[1])
positive_elements = set(pos_quant_tot.keys())
negative_elements = set(neg_quant_tot.keys())

In [7]:
# drop negative Qta rows that are not matched with positive ones
not_valid = negative_elements - (positive_elements & negative_elements)
indexes_todelete = []
for key in not_valid:
    for x in neg_quant_tot[key][1]:
        indexes_todelete.append(x)
df.drop(indexes_todelete, inplace=True)

In [8]:
# drop negative Qta rows that are invalid neg_qta > pos_qta
indexes_todelete = []
for key in positive_elements & negative_elements:
    if pos_quant_tot[key] < neg_quant_tot[key][0]:
            for x in neg_quant_tot[key][1]:
                indexes_todelete.append(x)
df.drop(indexes_todelete, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405448 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BasketID         405448 non-null  object 
 1   BasketDate       405448 non-null  object 
 2   Sale             405448 non-null  float64
 3   CustomerID       405448 non-null  object 
 4   CustomerCountry  405448 non-null  object 
 5   ProdID           405448 non-null  object 
 6   ProdDescr        405448 non-null  object 
 7   Qta              405448 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 18.6+ MB


In [10]:
# Create the dataframe for customers
data = pd.core.frame.DataFrame({'CustomerID' : df['CustomerID'].unique()})

In [11]:
unique_products = []
for customer in df["CustomerID"].unique():
    unique_products.append(len(df.loc[df["CustomerID"] == customer, ["ProdID"]]["ProdID"].unique()))

In [12]:
# I: the total number of items purchased by a customer during the period of observation
data['I'] = 0

In [13]:
# Iu: the number of distinct items bought by a customer in the period of observation.
data["Unique_Products"] = unique_products

In [14]:
# Imax: the maximum number of items purchased by a customer during a shopping session
data['Imax'] = 0

In [15]:
# E: the Shannon entropy on the purchasing behaviour of the customer
data['E'] = 0

In [16]:
# ProdPerOrderMean: mean number of items purchased by a customer during a shopping session

In [17]:
# SaleTot: total amount spent during the period of observation

In [18]:
# SaleTotWithoutReturn: total amount spent without negative quantity during the period of observation

In [19]:
# SaleMin: min amout spent during the shopping session

In [20]:
# SaleMeanPerOrder: mean amount spent for each order during the period of observation

In [21]:
# NumRetProd: number of returned products in the period of observation

In [22]:
# NumDistRetProd: number of distinct returned products in the period of observation

In [23]:
# MeanProdCostInOrder: mean cost of the products in the order

In [24]:
# OrderPerMonth

In [25]:
# ProductPerMonth

In [26]:
# PrizePerMonth

In [27]:
# OrderPerTrimester

In [28]:
# ProductPerTrimester

In [29]:
# PrizePerTrimester

In [30]:
# OrderPerTrimester 2-3

In [31]:
# ProductPerTrimester 2-3

In [32]:
# PrizePerTrimester 2-3

In [33]:
# CustomerCounty : value from original dataframe
#data['CustomerCountry'] = ''
#data = data.join(pd.core.frame.DataFrame({'CustomerID':df['CustomerID'], 'CustomerCountry':df['CustomerCountry']}).drop_duplicates().set_index('CustomerID'), on='CustomerID')

#tmp = pd.core.frame.DataFrame({'CustomerID':df['CustomerID'], 'CustomerCountry':df['CustomerCountry']}).drop_duplicates()
#tmp['CustomerID'].value_counts()[0:8]
# 4372 distinct customers - 4380 elements in tmp
# 8 customers have two different nationalities

# Find entries with multiple nationality
#counts = tmp['CustomerID'].value_counts().items()
#count_dict = {x[0]:x[1] for x in counts}
#for _, row in tmp.sort_values(by=['CustomerID']).iterrows():
#    if count_dict[row['CustomerID']] > 1:
#        print(f"{row['CustomerID']}-{row['CustomerCountry']}")

In [34]:
data

,CustomerID,I,Unique_Products,Imax,E
0,17850,0,21,0,0
1,13047,0,106,0,0
2,12583,0,115,0,0
3,13748,0,24,0,0
4,15100,0,1,0,0
...,...,...,...,...,...
4334,13436,0,12,0,0
4335,15520,0,18,0,0
4336,13298,0,2,0,0
4337,14569,0,10,0,0
